In [1]:
import numpy
from sklearn.datasets import load_boston
from sklearn.tree import DecisionTreeRegressor as decision_tree_regressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
class Node:
    def __init__(self, answer, father, condition = None, yes_son = None, no_son = None):
        self.answer = answer
        self.father = father
        self.condition = condition
        self.yes_son = yes_son
        self.no_son = no_son

        if self.father == None:
            self.depth = 0
        else:
            self.depth = father.depth + 1
        
    def satisfies(self, x):
        return self.condition(x)

    def next_node(self, x):
        return self.yes_son if self.satisfies(x) else self.no_son

def make_condition(split_feature, split_value):
    return (lambda x: x[split_feature] < split_value)

class DecisionTree:
    def __init__(self, stop_fraction=0.01, max_depth=numpy.inf, quantilies_number=9, uniform_thresholds_number=9):
        self.head = Node(None, None)
        self.stop_fraction = stop_fraction
        self.max_depth = max_depth
        self.quantilies_number = quantilies_number
        self.uniform_thresholds_number = uniform_thresholds_number

    def get_split_values(self, feature_number):
        values = numpy.copy(self.data[:, feature_number])
        values.sort()
        
        # Квантили
        thresholds = {
            values[int(level * len(values))] for level in numpy.linspace(0, 1, self.quantilies_number + 2)[1:-1]
        }
        
        # И просто несколько равномерно распределенных значений
        thresholds.update(
            numpy.linspace(numpy.min(values), numpy.max(values), self.uniform_thresholds_number + 2)[1:-1]
        )
        return thresholds

    def income_objects(self, node):
        path = []
        current_node = node
        while current_node != None:
            path.append(current_node)
            current_node = current_node.father
        path.reverse()

        result = []
        for values, target in zip(self.data, self.targets):
            gets_to_node = True
            for i in xrange(len(path) - 1):
                if path[i].next_node(values) != path[i + 1]:
                    gets_to_node = False
                    break
            if gets_to_node:
                result.append((values, target))
        return result

    def calculate_error(self, splitting_node, split_feature, split_value):
        income_objects = self.income_objects(splitting_node)
        yes_targets = numpy.array([
            target for values, target in income_objects
            if make_condition(split_feature, split_value)(values)
        ])
        no_targets = numpy.array([
            target for values, target in income_objects
            if not make_condition(split_feature, split_value)(values)
        ])

        yes_square_errors = numpy.array([
            (target - yes_targets.mean()) ** 2 for target in yes_targets
        ])

        no_square_errors = numpy.array([
            (target - no_targets.mean()) ** 2 for target in no_targets
        ])
            
        return ((
                yes_square_errors.mean() * len(yes_targets) / (len(yes_targets) + len(no_targets)) +
                no_square_errors.mean() * len(no_targets) / (len(yes_targets) + len(no_targets)),
                yes_targets.mean(),
                no_targets.mean()
            )
        )

    def node_fraction(self, node):
        return float(len(self.income_objects(node))) / len(self.data)        

    def split_node(self, node):
        if node.depth >= self.max_depth:
            return
        
        best_split_feature, best_split_value = None, None
        best_split_error = numpy.inf

        for feature_number in xrange(self.featuries_number):
            split_values = self.get_split_values(feature_number)
            for split_value in split_values:
                error = self.calculate_error(node, feature_number, split_value)[0]
                if error < best_split_error:
                    best_split_error = error
                    best_split_feature, best_split_value = feature_number, split_value
                
        error, yes_answer, no_answer = (
            self.calculate_error(node, best_split_feature, best_split_value)
        )

        yes_node = Node(yes_answer, node)
        no_node = Node(no_answer, node)
        node.yes_son = yes_node
        node.no_son = no_node
        node.condition = make_condition(best_split_feature, best_split_value)

        # Если в каком-то из детей будет слишком мало вершин, забьем на это разбиение.
        if (
            (self.node_fraction(yes_node) < self.stop_fraction) or
            (self.node_fraction(no_node) < self.stop_fraction)
        ):
            node.yes_son = None
            node.no_son = None
            node.condition = None
        else:
            self.split_node(yes_node)
            self.split_node(no_node)

    def fit(self, data, targets):
        self.data = data
        self.targets = targets
        self.featuries_number = len(data[0])
        self.split_node(self.head)

    def predict(self, values_array):
        result = []
        for values in values_array:
            current_node = self.head
            while current_node.yes_son != None:
                current_node = current_node.next_node(values)
            result.append(current_node.answer)
        return result

In [3]:
data = load_boston()
train_data, test_data, train_target, test_target = train_test_split(
    data['data'], data['target'], test_size=0.3, random_state=1
)

In [4]:
# Посмотрим качество своего дерева

my_tree = DecisionTree(max_depth=5)
my_tree.fit(train_data, train_target)
print mean_squared_error(my_tree.predict(test_data), test_target)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:85: RuntimeWarning: Mean of empty slice.
/home/frystile/.local/lib/python2.7/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:87: RuntimeWarning: Mean of empty slice.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:86: RuntimeWarning: Mean of empty slice.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:88: RuntimeWarning: Mean of empty slice.


16.6859902769


In [5]:
# И качество sklearn'овского дерева

sklearn_tree = decision_tree_regressor(max_depth=5)
sklearn_tree.fit(train_data, train_target)
print mean_squared_error(sklearn_tree.predict(test_data), test_target)

12.5828713978
